# Preparing

## Imports and Functions

In [1]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import regex as re
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

### Normal

In [2]:
folder_path = '/Users/Felix/Google Drive/Felix/code/AgeRecognition/raw_data/Faces'
#folder_path = '/content/UTKFaces/UTKFaces/Faces'

In [41]:
def load_images_from_folder(folder_path,percent=100):
    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    images = []
    for filename in filenames:
        img = cv2.imread(os.path.join(folder_path,filename),cv2.IMREAD_UNCHANGED)
        
        scale_percent = percent # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # resize image
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow("Resized image", resized)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        if img is not None:
            img_colors = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            images.append(img_colors)
    return images

def show_all_images_from_list(img_list,img_df):
    n=int(len(img_list)/5) +1 
    f = plt.figure(figsize=(25,n*5))
    for i,img in enumerate(img_list):
        f.add_subplot(n, 5, i + 1)
        plt.title(f'{img_df.iloc[i]}',
                  fontweight="bold",
                  fontsize=15)
        plt.imshow(img)
        f.tight_layout()

def load_image_data_into_dataframe(folderpath):

    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    df = pd.DataFrame()
    for filename in filenames:
        if filename.startswith('81'):
            break
        parsed_name = filename.split('_')
        df= df.append({'age' : int(parsed_name[0]),
                       'gender' : parsed_name[1],
                       'race' : parsed_name[2]}, ignore_index=True)
    df['gender'] = df['gender'].map({'0': 'male', '1': 'female'})
    df['race'] = df['race'].map({'0': 'White', '1': 'Black', '2': 'Asian', '3': 'Indian', '4': 'Others'})
    return df

In [29]:
img_pic=load_images_from_folder(folder_path)

In [30]:
img_df = load_image_data_into_dataframe(folder_path)

## Targets into Age Bins of 8 (Baseline: 10%)

In [31]:
bins= [8*i for i in range(11)]
bins

[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80]

In [32]:
labels = [0,1,2,3,4,5,6,7,8,9]
img_df['age_group'] = pd.cut(img_df['age'], bins=bins, labels=labels)

In [33]:
img_df

,age,gender,race,age_group
0,1.0,male,White,0
1,1.0,male,White,0
2,1.0,male,White,0
3,1.0,male,White,0
4,1.0,male,White,0
...,...,...,...,...
22000,80.0,female,Black,9
22001,80.0,female,Asian,9
22002,80.0,female,Asian,9
22003,80.0,female,Indian,9


In [34]:
img_df['age_group'].value_counts().sort_values()

9     533
8     843
1    1133
7    1137
5    1517
6    1920
2    2600
0    2817
4    3195
3    6310
Name: age_group, dtype: int64

## CNN 1 Preperations (Size: Original)

In [35]:
X = img_pic
y = img_df['age_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

(19804, 200, 200, 3)

In [36]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [37]:
#np.save('X_train', X_train)
#np.save('X_test', X_test)
#np.save('y_train_cat', y_train_cat)
#np.save('y_test_cat', y_test_cat)

In [38]:
print(X_train.shape)
print(X_test.shape)

(19804, 200, 200, 3)
(2201, 200, 200, 3)


## CNN 1 (Accuracy: 27,6 %)

In [39]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(200, 200, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(10, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [40]:
model = initialize_model()

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

Epoch 1/10
434/434 [==============================] - 146s 333ms/step - loss: 2.2307 - accuracy: 0.2888 - val_loss: 2.1731 - val_accuracy: 0.2790
Epoch 2/10
434/434 [==============================] - 104s 238ms/step - loss: 2.1301 - accuracy: 0.2918 - val_loss: 2.1145 - val_accuracy: 0.2790
Epoch 3/10
434/434 [==============================] - 103s 236ms/step - loss: 2.0875 - accuracy: 0.2918 - val_loss: 2.0930 - val_accuracy: 0.2790
Epoch 4/10
434/434 [==============================] - 94s 216ms/step - loss: 2.0718 - accuracy: 0.2918 - val_loss: 2.0867 - val_accuracy: 0.2790
Epoch 5/10
434/434 [==============================] - 92s 211ms/step - loss: 2.0666 - accuracy: 0.2918 - val_loss: 2.0850 - val_accuracy: 0.2790
Epoch 6/10
434/434 [==============================] - 91s 209ms/step - loss: 2.0647 - accuracy: 0.2918 - val_loss: 2.0847 - val_accuracy: 0.2790
Epoch 7/10
434/434 [==============================] - 92s 212ms/step - loss: 2.0639 - accuracy: 0.2918 - val_loss: 2.0847 - val

In [42]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.2757837474346161


## CNN Preperations Image Size: 100*100

In [43]:
img_pic=load_images_from_folder(folder_path,50)

In [44]:
X = img_pic
y = img_df['age_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

(19804, 100, 100, 3)

In [45]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [46]:
print(X_train.shape)
print(X_test.shape)

(19804, 100, 100, 3)
(2201, 100, 100, 3)


# CNN A

## CNN 2 (1 hidden with 10) 43,4%

In [57]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(10, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [58]:
model2 = initialize_model()
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model2.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

Epoch 1/10
434/434 [==============================] - 37s 84ms/step - loss: 1.8578 - accuracy: 0.3365 - val_loss: 1.6862 - val_accuracy: 0.3904
Epoch 2/10
434/434 [==============================] - 31s 72ms/step - loss: 1.5432 - accuracy: 0.4149 - val_loss: 1.5343 - val_accuracy: 0.4157
Epoch 3/10
434/434 [==============================] - 29s 68ms/step - loss: 1.4030 - accuracy: 0.4686 - val_loss: 1.4550 - val_accuracy: 0.4458
Epoch 4/10
434/434 [==============================] - 29s 67ms/step - loss: 1.3169 - accuracy: 0.5010 - val_loss: 1.4277 - val_accuracy: 0.4527
Epoch 5/10
434/434 [==============================] - 27s 62ms/step - loss: 1.2502 - accuracy: 0.5170 - val_loss: 1.4060 - val_accuracy: 0.4500
Epoch 6/10
434/434 [==============================] - 26s 60ms/step - loss: 1.2033 - accuracy: 0.5314 - val_loss: 1.4034 - val_accuracy: 0.4547
Epoch 7/10
434/434 [==============================] - 26s 59ms/step - loss: 1.1538 - accuracy: 0.5473 - val_loss: 1.4265 - val_accuracy:

In [59]:
print(model2.evaluate(X_test, y_test_cat, verbose=0)[1])

0.4343480169773102


## CNN 3 (1 hidden with 50 ) 45,6 %

In [60]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                510       
Total params: 922,552
Trainable params: 922,552
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 33s 76ms/step - loss: 1.6255 - accuracy: 0.4092 - val_

## CNN 4 (1 hidden with 100 )  46,5%

In [61]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_29 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_30 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 31s 70ms/step - loss: 1.6552 - accuracy: 0.4008 - 

## CNN 5 (2 hidden with 100,50 )  45,3%

In [62]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 39s 89ms/step - loss: 1.6326 - accuracy: 0.4055 - val_loss: 1.4644 - val_accuracy: 0.4354
Epoch 2/10
434/434 [==============================] - 27s 62ms/step - loss: 1.3458 - accuracy: 0.4796 - val_loss: 1.3871 - val_accuracy: 0.4534
Epoch 3/10
434/434 [==============================] - 27s 62ms/step - loss: 1.2347 - accuracy: 0.5141 - val_loss: 1.4053 - val_accuracy: 0.4490
Epoch 4/10
434/434 [==============================] - 26s 61ms/step - loss: 1.1454 - accuracy: 0.5504 - val_loss: 1.4324 - val_accuracy: 0.4647
Epoch 5/10
434/434 [==============================] - 26s 60ms/step - loss: 1.0361 - accuracy: 0.5956 - val_loss: 1.4529 - val_accuracy: 0.4461
Epoch 6/10
434/434 [==============================] - 27s 61ms/step - loss: 0.9449 - accuracy: 0.6325 - val_loss: 1.4941 - val_accuracy: 0.4504
Epoch 7/10
434/434 [==============================] - 26s 61ms/step - loss: 0.8470 - accuracy: 0.6702 - val_loss: 1.5151 - val_accuracy:

## CNN 6 (2 hidden with 100,100 ) 45,2%

In [63]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 31s 70ms/step - loss: 1.5683 - accuracy: 0.4185 - val_loss: 1.4176 - val_accuracy: 0.4488
Epoch 2/10
434/434 [==============================] - 28s 65ms/step - loss: 1.3106 - accuracy: 0.4864 - val_loss: 1.3446 - val_accuracy: 0.4663
Epoch 3/10
434/434 [==============================] - 28s 65ms/step - loss: 1.1936 - accuracy: 0.5270 - val_loss: 1.3717 - val_accuracy: 0.4631
Epoch 4/10
434/434 [==============================] - 27s 63ms/step - loss: 1.0730 - accuracy: 0.5753 - val_loss: 1.3739 - val_accuracy: 0.4589
Epoch 5/10
434/434 [==============================] - 29s 66ms/step - loss: 0.9493 - accuracy: 0.6244 - val_loss: 1.4551 - val_accuracy: 0.4679
Epoch 6/10
434/434 [==============================] - 29s 67ms/step - loss: 0.8280 - accuracy: 0.6738 - val_loss: 1.5430 - val_accuracy: 0.4478
Epoch 7/10
434/434 [==============================] - 28s 64ms/step - loss: 0.6904 - accuracy: 0.7342 - val_loss: 1.7176 - val_accuracy:

## CNN 7 (3 hidden with 50,100,50 )  46,4%

In [64]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_37 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_38 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_39 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_40 (Dense)             (None, 10)              

## CNN 8 (1 hidden with 300 )  46,4%

In [65]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(300, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 300)               5529900   
_________________________________________________________________
dense_42 (Dense)             (None, 10)                3010      
Total params: 5,533,302
Trainable params: 5,533,302
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 36s 82ms/step - loss: 1.5864 - accuracy: 0.4217 - 

## CNN 9 (2 hidden with 100,20 )  47,3%

In [66]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_43 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_44 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_45 (Dense)             (None, 10)                210       
Total params: 1,845,922
Trainable params: 1,845,922
Non-trainable params: 0
___________________________________________

## CNN 10 (3 hidden with 20, 50, 20 )  45,6%

In [67]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(20, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Third fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_46 (Dense)             (None, 20)                368660    
_________________________________________________________________
dense_47 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_48 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_49 (Dense)             (None, 10)              

# CNN B

## CNN 1 (1 hidden with 100) - (Epochs:100, Patience: 10)   46,6%
**High_Acc - low Val_Acc**

In [68]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_50 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_51 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
434/434 [==============================] - 32s 73ms/step - loss: 1.6793 - accuracy: 0.3927 -

## CNN 1.2 - Dropout, Hidden Layer 50% => 46,4%

In [69]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.5))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_52 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.3 Dropout First and Hidden Layer 20%    45,5 %

In [70]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(Dropout(0.2, input_shape=(100,100,3,)))
model.add(layers.Conv2D(8, (4,4), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_54 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 10)              

## CNN 1.4 Dropout, Hidden Layer 40% => 47,4%

In [71]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.4))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_56 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.5 Dropout, Hidden Layer 30% => 46,7%

In [72]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.3))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 1.6 Dropout, Hidden Layer 20% => 46,7%

In [73]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### Hidden Layers
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(10, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_60 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 10)                1010      
Total params: 1,844,702
Trainable params: 1,844,702
Non-trainable params: 0
___________________________________________

## CNN 2 Something new (Copy from Cifar Classification) 48,5 %

In [74]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 100, 100, 32)      896       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 25, 25, 128)     

In [75]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.48523399233818054
